# Welcome to the [NFL Big Data Bowl 2022](https://www.kaggle.com/c/nfl-big-data-bowl-2022)!

<center><img src='https://www.stadiumsofprofootball.com/wp-content/uploads/2016/07/metlifegiants2012955.jpg' width='600' height="600" ></center>

#### This notebook will go through an extensive analysis to view and explore the NFL Big Data Bowl 2022 competition data , this note bote (for now) will be focusing on 2018 data to explore the plays data and players movements, to that end we will be creating functions to easilty plot players movments
<h4><font color='darkred'>Please note that this work you're about to see here is a work in progress so a lot will be added later</font></h4>


## The notebook will have the following sections:
 ### - [Data Imports and initial exploring](#load_data).
 ####  you can skip this part of you are familiar with the competition data.
 ### - [Exploratory data analysis](#EDA).
 ### - [Player movement analysis](#PMEDA).
 ### - [ML starter](#ml).
 ####  this part will contain the whole EDA process to explore all the datasets.
#### P.S.  There won't be a dedicated parts for cleaning or features engineering as few of both will be done along the way since the data is pretty much preprocessed.

### First we import libraries to work with

In [ ]:
# imports
import gc
import os
import sys
from io import BytesIO
import time
from IPython.display import Image, display as image, display
from IPython.display import HTML
import requests
from PIL import Image
import numpy as np
import statistics as st
import datetime
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import rc
import pandas as pd
import dask.dataframe as dd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows',120)
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.gridspec as gridspec
import seaborn as sns
from termcolor import colored
%matplotlib inline
sns.set_style('darkgrid')
import plotly.express as px
import plotly.figure_factory as ff
hcmap = sns.blend_palette(['yellowgreen','orange','orangered','red'],n_colors=12,as_cmap=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.feature_selection import mutual_info_regression
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split as split
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV as Grid
import xgboost as xgb
from sklearn.metrics import (f1_score, accuracy_score, balanced_accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix)

<a id='load_data'></a>
# Loading Data and exploring main statistics

### This part will be dedicated to data imports and also initial exploring of the datasets.

In [ ]:
input_data_path = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        input_data_path.append(os.path.join(dirname, filename))

In [ ]:
input_data_path

In [ ]:
games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/games.csv')
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/plays.csv')
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/players.csv')
tracking2018 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv',parse_dates=True)
tracking2019 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2019.csv',parse_dates=True)
tracking2020 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2020.csv',parse_dates=True)
PFFScoutingData = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
temperature = pd.read_csv('/kaggle/input/nfl-temp/temptreture.csv')

<center><h2> Now let the analysis begins</h2></center>


<center><img src="https://www.reactiongifs.us/wp-content/uploads/2018/11/2018-NFL-Football.gif" width='600' height="600"></center>



### Checking tracking datasets

In [ ]:
tracking2018.head(2)

### Checking if all columns across all year datasets all equal

In [ ]:
list(tracking2018.columns) == list(tracking2019.columns) == list(tracking2020.columns)

<center><img src='https://c.tenor.com/W36eIGSnVakAAAAC/dragons-den-very-good.gif'></center>

### Getting values with null nflId's

In [ ]:
tracking2018[tracking2018.nflId.isna()].sample(5)

In [ ]:
tracking2018[tracking2018.nflId.isna()].displayName.unique()

> #### Seems like all null values are associated with the ball 🏈 tracking

### Separating ball tracking from players tracking data

In [ ]:
ball_18_df = tracking2018[tracking2018.team == 'football'].copy()
tracking2018 = tracking2018[tracking2018.team != 'football'].copy()

ball_19_df = tracking2019[tracking2019.team == 'football'].copy()
tracking2019 = tracking2019[tracking2019.team != 'football'].copy()

ball_20_df = tracking2020[tracking2020.team == 'football'].copy()
tracking2020 = tracking2020[tracking2020.team != 'football'].copy()

ball_18_df.drop(columns=['o', 'dir', 'nflId', 'displayName', 'jerseyNumber', 'position'],inplace=True)
ball_19_df.drop(columns=['o', 'dir', 'nflId', 'displayName', 'jerseyNumber', 'position'],inplace=True)
ball_20_df.drop(columns=['o', 'dir', 'nflId', 'displayName', 'jerseyNumber', 'position'],inplace=True)
gc.collect()

ball_20_df.sample(4)

### Editing some data types to save memory

In [ ]:
Dtypes = {'time':'datetime64[ns]', 'x':'float32', 'y':'float32', 's':'float32', 'a':'float32',
          'dis':'float32', 'o':'float32', 'dir':'float64','event':'object' , 'nflId':'uint16',
          'displayName':'object', 'jerseyNumber':'uint8', 'position':'object', 'team':'object',
          'frameId':'int32', 'gameId':'uint32', 'playId':'int16'}

ball_dtypes = {'time':'datetime64[ns]', 'x':'float32', 'y':'float32', 's':'float32', 'a':'float32',
               'dis':'float32','event':'object' ,'frameId':'int32','gameId':'uint32','playId':'int16'}

### Now we will be converting data types to save memory

In [ ]:
tracking2018 = tracking2018.astype(Dtypes).copy()
tracking2019 = tracking2019.astype(Dtypes).copy()
tracking2020 = tracking2020.astype(Dtypes).copy()

tracking2018 = tracking2018.sort_values(by=['time','gameId','playId', 'displayName']).copy()
tracking2019 = tracking2019.sort_values(by=['time','gameId','playId', 'displayName']).copy()
tracking2020 = tracking2020.sort_values(by=['time','gameId','playId', 'displayName']).copy()

ball_18_df = ball_18_df.astype(ball_dtypes).copy()
ball_19_df = ball_19_df.astype(ball_dtypes).copy()
ball_20_df = ball_20_df.astype(ball_dtypes).copy()

ball_18_df = ball_18_df.sort_values(by=['time','gameId', 'playId']).copy()
ball_19_df = ball_19_df.sort_values(by=['time','gameId', 'playId']).copy()
ball_20_df = ball_20_df.sort_values(by=['time','gameId', 'playId']).copy()

gc.collect()

#### Actually using .copy() and the above mentioned data types helpes reducing the memory usage dramatically (down somewhere around 6.2gb), I think this is as good as it gets if you wanted to go with pandas

### Now we explore `players` DataFrame

In [ ]:
players.head(3)

### Getting the names of players with missing birth year

In [ ]:
players.birthDate.isna().sum()

In [ ]:
players[players.birthDate.isna()]

In [ ]:
missin_dates = {'Antoine Brooks':'1997-10-28', 'Bradlee Anae': '1998-10-17', 'Bravvion Roy':'1996-10-18',
                'Bryce Hall': '1999-08-14', 'Carter Coughlin': '1997-07-21', 'Chris Claybrooks': '1998-07-17',
                'Derrek Tuszka':'1997-08-17', 'Hunter Niswander': '1994-11-26', 'James Smith-Williams': '1997-07-29',
                'Jameson Houston': '1996-06-30', 'Kamren Curl': '1999-03-31', 'Keandre Jones': '1997-09-24',
                'Michael Danna':  '1997-12-4','Raekwon Davis': '1997-06-10', 'Rashard Lawrence': '1998-08-27',
                'Taylor Russolino':'1989-05-23', 'Trevon Diggs':'1997-09-20'}

### Adding birth dates

In [ ]:
missing_index = players[players.birthDate.isna()].index
for i in missing_index:
    plyr = players.loc[i,'displayName'] 
    date = missin_dates[plyr]
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    players.loc[i, 'birthDate'] = date

In [ ]:
players.loc[missing_index,:]

### Now we extrac heights of players and convert it to centimeters and also will be extracting age of players

In [ ]:
players['feets'] = players.height.str.extract(r'^(\d)\-').astype(float)
players['inchs'] = players.height.str.extract(r'(^\d\d)').astype(float)
players['alsoinchs'] = players.height.str.extract(r'\-(\d*)').astype(float)
players.fillna(0,inplace=True)
players['highcm'] = (players['feets'] * 30.48) + (players['inchs'] *2.54) + (players['alsoinchs']*2.54)
players['age'] = 2018 - pd.to_datetime(players.birthDate).dt.year
players.drop(columns=['feets','inchs','alsoinchs'],inplace=True)
gc.collect()
players.sample(4)

In [ ]:
players.info()

### Checking dublicated name (it happens!)

In [ ]:
players[players.displayName.duplicated(keep=False)]

### Checking if these names related to the previous missing birth dates

In [ ]:
set(players[players.displayName.duplicated(keep=False)].index).intersection(set(missing_index))

### Getting the duplicated names

In [ ]:
duplicated_names = sorted(list(set(players[players.displayName.duplicated(keep=False)].displayName.values)))

### checking if all players in tracking are in the players dataset

In [ ]:
set(tracking2018.displayName.unique()).issubset(set(players.displayName.unique()))

In [ ]:
set(tracking2018.displayName.unique()).difference(set(players.displayName.unique()))

### Getting height and weight (manually)

In [ ]:
missing_players = {'Charles Leno Jr.':[193 ,304],'Chris Herndon':[193 ,253], 'Chris Hubbard':[193, 295 ],
                   'Dan McCullers':[201,348], 'Danny Vitale':[183, 225], 'David Williams':[191, 294 ],
                   'DeVante Bausby':[185 , 190 ], 'Domata Peko Sr.':[191, 322 ], "Donte' Deayon":[175, 159],
                   'Foye Oluokun':[188 ,215], 'Jake Martin':[188,242  ], 'Jeff Wilson':[183,213],
                   'Joe Jones':[183 , 240], 'Joe Noteboom':[196, 260 ], 'Jon Weeks':[178, 242],
                   'Josh Perkins':[190, 227], "Julie'n Davenport":[201, 313], 'KhaDarel Hodge':[188, 205],
                   'L.P. LaDouceur':[196, 255], 'Lac Edwards':[193, 209], 'Matt McCrane':[175, 165],
                   'Matthew Ioannidis':[190,310], 'Matthew Slater':[183, 205], 'Mike Badgley':[178, 183],
                   'Nate Gerry':[188, 230],'Nickell Robey-Coleman':[173, 180], 'Shaq Mason':[185, 310],
                   'Trent Brown':[203, 380], 'Trent Scott':[196, 320], 'Zeke Turner':[188,214]}

In [ ]:
games.head(3)

### Checking if there is any duplication in gameid in games dataset

In [ ]:
games.gameId.duplicated().sum()

### Setting games DataFrame index to gameId

In [ ]:
games.set_index('gameId',inplace=True)
games.gameDate = games.gameDate.astype('datetime64[ns]')
games.head(8)

### Checking if all games in tracking DataFrame are included in the games DataFrame

In [ ]:
set(tracking2018.gameId).issubset(set(games.index))

### Mapping teams abbreviations to full name (got that from [NFL 2021 Rulebook](https://operations.nfl.com/the-rules/2021-nfl-rulebook/#team-abbreviation-codes))
### We will also map a team name to NOAA weather stations to get weather data of games.

In [ ]:
team_abb = {'ARI': ['Arizona Cardinals','PHOENIX DEER VALLEY MUNICIPAL AIRPORT, AZ US','NFC', 'west'],
            'ATL': ['Atlanta Falcons','ATLANTA FULTON CO AIRPORT, GA US','NFC', 'south'],
            'BAL': ['Baltimore Ravens', 'BALTIMORE WASHINGTON INTERNATIONAL AIRPORT, MD US','AFC', 'north'],
            'BUF': ['Buffalo Bills','BUFFALO NIAGARA INTERNATIONAL, NY US','AFC', 'east'],
            'CAR': ['Carolina Panthers','CHARLOTTE DOUGLAS AIRPORT, NC US','NFC', 'south'],
            'CHI': ['Chicago Bears', 'CHICAGO MIDWAY AIRPORT, IL US','NFC', 'north'],
            'CIN': ['Cincinnati Bengals', 'CINCINNATI MUNICIPAL AIRPORT LUNKEN FIELD, OH US','AFC', 'north'],
            'CLE': ['Cleveland Browns','CLEVELAND BURKE LAKEFRONT AIRPORT, OH US','AFC', 'north'],
            'DAL': ['Dallas Cowboys','DALLAS REDBIRD AIRPORT, TX US','NFC', 'east'],
            'DEN': ['Denver Broncos','DENVER WATER DEPARTMENT, CO US','AFC', 'west'],
            'DET': ['Detroit Lions', 'DETROIT CITY AIRPORT, MI US','NFC', 'north'],
            'GB': ['Green Bay Packers','GREEN BAY A S INTERNATIONAL AIRPORT, WI US','NFC', 'north'],
            'HOU': ['Houston Texans', 'HOUSTON WILLIAM P HOBBY AIRPORT, TX US','AFC', 'south'],
            'IND': ['Indianapolis Colts', 'INDIANAPOLIS NWFO, IN US','AFC', 'south'],
            'JAX': ['Jacksonville Jaguars','JACKSONVILLE CRAIG MUNICIPAL AIRPORT, FL US','AFC', 'south'],
            'KC': ['Kansas City Chiefs', 'KANSAS CITY DOWNTOWN AIRPORT, MO US','AFC', 'west'],
            'LV': ['Las Vegas Raiders', 'LAS VEGAS AIR TERMINAL, NV US','AFC', 'west'],
            'LAC': ['Los Angeles Chargers','WHITTIER HILLS CALIFORNIA, CA US','AFC', 'west'],
            'LA': ['Los Angeles Rams','WHITTIER HILLS CALIFORNIA, CA US','NFC', 'west'],
            'MIA': ['Miami Dolphins','MIAMI OPA LOCKA AIRPORT, FL US','AFC', 'east'],
            'MIN': ['Minnesota Vikings','UNIVERSITY OF MN ST. PAUL, MN US','NFC', 'north'],
            'NE': ['New England Patriots', 'NORWOOD MEMORIAL AIRPORT, MA US','AFC', 'east'],
            'NO': ['New Orleans Saints','NEW ORLEANS AUDUBON, LA US','NFC', 'south'],
            'NYG': ['New York Giants','JFK INTERNATIONAL AIRPORT, NY US','NFC', 'east'],
            'NYJ': ['New York Jets','JFK INTERNATIONAL AIRPORT, NY US','AFC', 'east'],
            'OAK': ['Oakland Raiders', 'OAKLAND MUSEUM, CA US','AFC', 'west'],
            'PHI': ['Philadelphia Eagles','PHILADELPHIA INTERNATIONAL AIRPORT, PA US','NFC', 'east'],
            'PIT': ['Pittsburgh Steelers','PITTSBURGH ASOS, PA US','AFC', 'north'],
            'SF': ['San Francisco 49ers','SAN JOSE INTERNATIONAL AIRPORT, CA US','NFC', 'west'],
            'SEA': ['Seattle Seahawks', 'SEATTLE SAND POINT WEATHER FORECAST OFFICE, WA US','NFC', 'west'],
            'TB': ['Tampa Bay Buccaneers', 'TAMPA INTERNATIONAL AIRPORT, FL US','NFC', 'south'],
            'TEN': ['Tennessee Titans', 'NASHVILLE INTERNATIONAL AIRPORT, TN US','AFC', 'south'],
            'WAS': ['Washington Football Team','OXON HILL, MD US','NFC', 'east']}

### Now we explore the temperature data collected from the NOAA

In [ ]:
temperature.sample(8)

In [ ]:
temperature = temperature.astype({'DATE':'datetime64[ns]'}).copy()
temperature['month'] = temperature.DATE.dt.month
temperature['city'] = temperature.NAME.str.extract(r'(^\w+\s\w+)\W')
temperature['STATE'] = temperature.NAME.str.extract(r'\W(\w{2})\s\w\w$')
temperature.sort_values(by=['NAME','DATE'],inplace=True)
temperature = temperature[temperature.month.isin([12, 11, 10, 9, 1])].copy()
temperature.TMAX.fillna(method= 'ffill',inplace=True)
temperature.TMIN.fillna(method= 'ffill',inplace=True)
temperature.TAVG.fillna(temperature[['TMAX','TMIN']].mean(axis=1),inplace=True)
temperature.sample(4)

### Finaly we merge temperature dataframe with the games one

### First we make a key from the games dataframe

In [ ]:
games['home_key'] = games.homeTeamAbbr.apply(lambda x: team_abb[x][1])

In [ ]:
games = games.reset_index().merge(temperature, left_on= ['gameDate','home_key'],how='left', right_on=['DATE','NAME']).copy()
games = games[['gameId','season', 'week', 'gameDate', 'gameTimeEastern', 'homeTeamAbbr',
                 'visitorTeamAbbr', 'home_key', 'TAVG', 'month', 'STATE']].copy()
games.month = games.gameDate.dt.month_name()
games['home_conference'] = games.homeTeamAbbr.apply(lambda x: team_abb[x][-2])
games['away_conference'] = games.visitorTeamAbbr.apply(lambda x: team_abb[x][-2])
games['home_region'] = games.homeTeamAbbr.apply(lambda x: team_abb[x][-1])
games['away_region'] = games.visitorTeamAbbr.apply(lambda x: team_abb[x][-1])
games['home_division'] = games.home_conference + ' ' + games.home_region
games['away_division'] = games.away_conference + ' ' + games.away_region
games[games.TAVG.isna()]

### Getting a list of states

In [ ]:
games.STATE.unique()

### Now we make a mapping for states timezone (only the difference from EST)

In [ ]:
state_time = {'PA': 0 , 'MD': 0, 'OH': 0, 'IN': 0, 'FL': 0, 'MA': 0, 'LA': -1, 'NY': 0, 'AZ': -2, 'NC': 0, 'WI': -1,
              'MI': 0, 'CA': -3, 'GA': 0, 'TN': -1, 'CO': -2, 'TX': -1, 'IL': -1, 'MO': -1, 'MN': -1, 'WA': -3,'NV': -3}
games['time_zone_adj'] = games.STATE.apply(lambda x: state_time[x])
games['gameTimeLocal'] = games.gameTimeEastern.apply(lambda x: int(x[:2])) + games.time_zone_adj
games['gameTimeLocal'] = games['gameTimeLocal'].astype(str) + games.gameTimeEastern.apply(lambda x: x[2:-3]).copy()
games.drop(columns='time_zone_adj', inplace=True)

In [ ]:
games.head()

### Now we check plays and PFFScoutingData datasets

In [ ]:
PFFScoutingData.head(2)

In [ ]:
plays.head(2)

> ### `playDescription` columns seems to be interesting as it have a text describing the play

### Adding `yards to end zone` feature to the plays dataframe

In [ ]:
plays['yards_to_endzone'] = np.select([plays.possessionTeam == plays.yardlineSide,plays.possessionTeam != plays.yardlineSide],
          [((plays.possessionTeam == plays.yardlineSide).astype(int) * (100 - plays.yardlineNumber)), plays.yardlineNumber], default=50)

### Engineering PreSnapscores features to show the score of team in possession and the receiving team in addtion to the score difference instead of only dealing with home and visitor labels

In [ ]:
temp_plays = plays.merge(games, left_on= 'gameId', right_on='gameId')
plays['pos_pre_score'] = np.select([(temp_plays.possessionTeam == temp_plays.homeTeamAbbr),(temp_plays.possessionTeam == temp_plays.visitorTeamAbbr)],[temp_plays.preSnapHomeScore, temp_plays.preSnapVisitorScore], default= 100000)
plays['def_pre_score'] = np.select([(temp_plays.possessionTeam != temp_plays.homeTeamAbbr),(temp_plays.possessionTeam != temp_plays.visitorTeamAbbr)],[temp_plays.preSnapHomeScore, temp_plays.preSnapVisitorScore], default= 100000)
plays['pos_score_def'] = plays.pos_pre_score - plays.def_pre_score
del temp_plays
plays.head()

<a id='EDA'></a>
# Exploratory Data Analysis

## This part will go through a complete EDA to explore all datasets
### We will be exploring the following datasets
* #### [Games](#gms).
* #### [Players](#players).
* #### [plays](#plays).
* #### [PFFScoutingData](#Epff).
* #### [Tracking data](#trk). 

<a id='gms'></a>

## First we explore games dataset

### We start first by showing games played inb each state given the conference

In [ ]:
plt.figure(figsize=(14,5))
sns.countplot(data=games, x= 'STATE',hue='home_conference',palette=['blue','red'], alpha=.8)
plt.title('Games played in each state given the conference',fontsize=18)
plt.legend(title='Conference', title_fontsize=12)
plt.tick_params(labelsize=14)
plt.ylabel('')
plt.xlabel('');

### Now we check game time

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(data=games, x='gameTimeEastern',color='darkblue',
              order=sorted(games.gameTimeEastern.unique()), alpha=.7)
plt.title('Games Starting time (EST)',fontsize=18)
plt.legend(title='Conference', title_fontsize=12)
plt.tick_params(labelsize=12,rotation=290)
plt.yticks(rotation=0)
plt.ylabel('')
plt.xlabel('');

In [ ]:
plt.figure(figsize=(18,6))
sns.countplot(data=games, x='gameTimeLocal',color='darkblue',
              order=sorted(games.gameTimeLocal.unique()), alpha=.7)
plt.title('Games Starting time (local time)',fontsize=18)
plt.tick_params(labelsize=12,rotation=290)
plt.yticks(rotation=0)
plt.ylabel('')
plt.xlabel('');

### Now we get the average temperature of air in each week

In [ ]:
plt.figure(figsize=(22,6))
sns.pointplot(data=games, x='week',y='TAVG', palette='viridis_r', hue='season', dodge=.3,markers= '^', linestyles=':')#,ci=99.9, palette='icefire_r', alpha= .8)
plt.ylabel('Average Temperature in F',fontsize=15)
plt.xlabel('Week', fontsize= 15)
plt.title('Average air temperature for each game week',fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=12);

<a id='players'></a>
## Exploring players dataset

### First we show the number of each player in each position (as this may afffect the overall avarage of any players statistics)

In [ ]:
fig = plt.figure(figsize=(16,6))
ax = fig.add_subplot(111)
sns.countplot(data=players, x ='Position',ax=ax, palette='viridis', order=players.Position.value_counts().index, alpha=.9)
plt.title('Counts of each player in every position', fontsize=22)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('')
plt.xlabel('');

### now we explore Players' heights and weights
### We start with players heights

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
sns.kdeplot(data=players, x ='highcm', linewidth=0,fill=True,alpha=.4)
plt.title("Players' heights KDE plot", fontsize=16);

> ### Even if hegihts look smooth it still looks like is a mixture of gaussians distributions 

### Showing a box plot of height for each player position

In [ ]:
fig = plt.figure(figsize=(20,7))
ax = fig.add_subplot(111)
sns.boxplot(data=players, y = 'highcm', x='Position',order= players.groupby('Position')['highcm'].mean().sort_values(ascending=False).index, ax=ax, palette='viridis')
plt.title("Players' heights for every position", fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('')
plt.xlabel('');

### The next figure is just to get a deeper look into each position distribution and how it differs from the overall mean

In [ ]:
fig = plt.figure(figsize=(22,26))
grid =  gridspec.GridSpec(7,4,figure=fig,hspace=.35,wspace=.2)
positions_and_means = list(zip(players.groupby('Position')['highcm'].mean().index,players.groupby('Position')['highcm'].mean()))
height_mean = players.highcm.mean() 
n =0
for i in range(7):
    for j in range(4):
        ax = fig.add_subplot(grid[i, j])
        pos = positions_and_means[n][0]
        sns.kdeplot(data= players[players.Position == pos ], x='highcm',ax=ax, color='darkblue',alpha=.4,linewidth=0,fill=True)
        ax.axvline(positions_and_means[n][1],label='Pos μ= {}'.format(round(positions_and_means[n][1],2)),color = 'orangered',linestyle ='--',alpha=.7)
        ax.axvline(height_mean,label='Ovr μ= {}'.format(round(height_mean,2)), color='blue',linestyle ='--',alpha=.7)
        ax.set_title('Position: '+positions_and_means[n][0],fontsize=14)
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_xlim(167,206)
        plt.legend(loc='upper left',fontsize= 10)
        n += 1
        if n ==26:
            break
fig.suptitle('Kde plot of players\' heights for every position', fontsize=16,y=.9);

### Now we check players weights in the very same way

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
sns.kdeplot(data=players, x ='weight', linewidth=0,fill=True,alpha=.4,color='darkblue')
plt.title("Players' weight KDE plot", fontsize=16);

> ### weights distribution seems a little different from heights as it looks like some kind of a trimodal distribution

In [ ]:
fig = plt.figure(figsize=(20,7))
ax = fig.add_subplot(111)
sns.boxplot(data=players, y = 'weight', x='Position',order= players.groupby('Position')['weight'].mean().sort_values(ascending=False).index, ax=ax, palette='viridis')
plt.title("Players' weight for every position", fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('')
plt.xlabel('');

In [ ]:
fig = plt.figure(figsize=(22,26))
grid =  gridspec.GridSpec(7,4,figure=fig,hspace=.35,wspace=.2)
positions_and_means = list(zip(players.groupby('Position')['weight'].mean().index,players.groupby('Position')['weight'].mean()))
weight_mean = players.weight.mean() 
n =0
for i in range(7):
    for j in range(4):
        ax = fig.add_subplot(grid[i, j])
        pos = positions_and_means[n][0]
        sns.kdeplot(data= players[players.Position == pos ], x='weight',ax=ax, color='darkblue',alpha=.4,linewidth=0,fill=True)
        ax.axvline(positions_and_means[n][1],label='Pos μ= {}'.format(round(positions_and_means[n][1],2)),color = 'orange',linestyle ='--',alpha=.7)
        ax.axvline(weight_mean,label='Ovr μ= {}'.format(round(weight_mean,2)), color='blue',linestyle ='--',alpha=.7)
        ax.set_title('Position: '+positions_and_means[n][0],fontsize=14)
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_xlim(153,380)
        plt.legend(loc='upper right',fontsize= 10)
        n += 1
        if n ==26:
            break
fig.suptitle('Kde plot of players\' weights for every position', fontsize=16,y=.9);

> ### Weights seems to differ from the overall mean in a more obvious way from heights

### The next figure shows the overall relation between height and weight

In [ ]:
g = sns.jointplot(data=players, y = 'weight', x='highcm',kind='reg',height=8,ratio=8,space=.1)
g.set_axis_labels(xlabel='Height (cm)', ylabel= 'Weight (lb)',fontsize=14);

<a id='plays'></a>
## Exploring the plays data sets

### First we get a summary of the plays dataset

In [ ]:
fig = plt.figure(figsize=(16,12))
grid =  gridspec.GridSpec(2,2 ,figure=fig, hspace=.1,wspace=.15 )
ax1 = fig.add_subplot(grid[0, 0])
ax2 = fig.add_subplot(grid[0, 1])
colors= [( 1.00, .682, .102),
         (.455, .569, .914),
         (0.477504, 0.821444, 0.318195),
         (0.134692, 0.658636, 0.517649)]
patches, texts, nums  = ax1.pie(x= plays.specialTeamsPlayType.value_counts()
                      , wedgeprops=dict(width=0.55,linewidth= 0),startangle=305,
                      center=(.2,.5),radius=1.1,colors=colors,autopct='%.1f%%',
                      textprops=dict(fontsize=14),)
ax1.legend(patches, plays.specialTeamsPlayType.value_counts().index, bbox_to_anchor=[0.1, 0.25],fontsize=11, title='Play type', title_fontsize=13)

ax1.set_title('Count of each play type', fontsize=18)
sns.kdeplot(data=plays[~plays.specialTeamsPlayType.isin(['Kickoff','Extra Point'])],
            x='yards_to_endzone',hue='specialTeamsPlayType',fill=True,linewidth=0,palette=['orange','darkblue'], ax= ax2,alpha=.5)
ax2.set_title('play type and yards to endzone', fontsize=18)
ax1.set_ylabel('')
ax2.set_ylabel('')
ax1.set_xlabel('', fontsize=14)
ax2.set_xlabel('',fontsize=14)

ax3 = fig.add_subplot(grid[1, 0])
ax4 = fig.add_subplot(grid[1, 1])

sns.scatterplot(data= plays, x='yards_to_endzone', y='kickLength',hue='specialTeamsPlayType',s=110,alpha=.4, ax=ax3,palette='viridis')

sns.kdeplot(data= plays[(plays.specialTeamsPlayType.isin(['Field Goal','punt']))],palette= 'viridis',
            hue='specialTeamsResult', x='pos_score_def',fill=True, alpha=.7, ax= ax4,
            hue_order=list(plays[(plays.specialTeamsPlayType.isin(['Field Goal','punt']))].specialTeamsResult.value_counts().sort_values().index))
ax3.legend(fontsize=11, title='Play type', title_fontsize=13)
ax3.set_ylabel('kick Length', fontsize=14)
ax3.set_xlabel('yards to end zone', fontsize=14)
ax4.set_ylabel('')
ax4.set_xlabel('Pos team score differnce',fontsize=14)
fig.suptitle('Plays dataset summary', y=.94, fontsize=22);

### The next figure shows a summary of each special team play result

In [ ]:
fig = plt.figure(figsize=(17,12))
grid =  gridspec.GridSpec(2,2 ,figure=fig, hspace=.2,wspace=.4 )

ax1 = fig.add_subplot(grid[0, 0])
ax2 = fig.add_subplot(grid[0, 1])
ax3 = fig.add_subplot(grid[1, 0])
ax4 = fig.add_subplot(grid[1, 1])
for t,x in zip(plays.specialTeamsPlayType.unique(), (ax1,ax2,ax3,ax4)):
    sns.countplot(data=plays[plays.specialTeamsPlayType == t], y='specialTeamsResult',
                  order= plays[plays.specialTeamsPlayType == t].specialTeamsResult.value_counts().index,color='darkblue',alpha=.6, ax=x)
    x.set_title('Results of {} plays'.format(t),fontsize=16)
    x.set_ylabel('')
    x.set_xlabel('')
    x.tick_params(axis='y', labelsize=11)
    x.tick_params(axis='x', labelsize=13)
fig.suptitle('Result summary of  each kind of Special teams plays', fontsize= 20, y=.95);

### Now we take a deeper look into yards to end zone special teams plays 
#### P.s we will explore Punts and Field goals only since kick offs and extra points have fixed starting yardage

In [ ]:
fig = plt.figure(figsize=(18, 7))
grid =  gridspec.GridSpec(1, 2, figure=fig, hspace=.18,wspace=.1 )
ax1 = fig.add_subplot(grid[0, 0])
ax2 = fig.add_subplot(grid[0, 1:])

punt_l = list(plays[(plays.specialTeamsPlayType.isin(['Punt']))].specialTeamsResult.value_counts().sort_values(ascending=True).index)
sns.kdeplot(data=plays[(plays.specialTeamsPlayType.isin(['Punt']))],
            x='yards_to_endzone',hue='specialTeamsResult',hue_order=punt_l,fill=True,linewidth=1,palette= 'viridis', alpha=.7, ax=ax1)
fgoal_l = ['Kick Attempt Good','Kick Attempt No Good','Non-Special Teams Result', 'Blocked Kick Attempt']
sns.kdeplot(data=plays[(plays.specialTeamsPlayType.isin(['Field Goal'])) & (plays.specialTeamsResult.isin(fgoal_l))],
            x='yards_to_endzone',hue='specialTeamsResult',hue_order=fgoal_l[::-1],fill=True,linewidth=1,palette='viridis', alpha=.7, ax=ax2)
ax1.set_title('Punts results distributions', fontsize=16)
ax2.set_title('Field goal results distributions', fontsize=16)
fig.suptitle('Yards to end zone and special teams plays results distributions', fontsize=22)
ax1.set_ylabel('')
ax2.set_ylabel('');

<a id='Epff'></a>
### Exploring the PFFScoutingData dataset

In [ ]:
fig = plt.figure(figsize=(16,12))
grid =  gridspec.GridSpec(2,2 ,figure=fig, hspace=.2,wspace=.15 )
ax1 = fig.add_subplot(grid[0, 0])
ax2 = fig.add_subplot(grid[0, 1])
ax3 = fig.add_subplot(grid[1, 0:])

sns.countplot(data= PFFScoutingData, x = 'kickDirectionIntended',
              hue= 'kickDirectionActual', palette='viridis', ax= ax1)
ax1.set_title('Count of kick  given the direction (intended vs real)', fontsize=16)
ax1.set_ylabel('')
ax1.set_xlabel('direction intended', fontsize=14)
ax1.legend(loc= 'upper right', fontsize= 10, title= 'Actual direction', title_fontsize=12)
sns.countplot(data= PFFScoutingData, x = 'kickType',
              hue= 'kickDirectionActual', palette='viridis', ax= ax2)
ax2.set_title( 'Count of each kick type given the direction', fontsize=16)
ax2.set_ylabel('')
ax2.set_xlabel('')
ax2.legend(loc= 'upper right', fontsize= 10, title= 'Kick direction', title_fontsize=12)
sns.kdeplot(data= PFFScoutingData,  hue= 'kickType',x= 'hangTime',hue_order= list(PFFScoutingData.kickType.value_counts().sort_values().index),
            palette='colorblind', fill=True, ax= ax3, alpha=.7)
ax3.set_title('KDE plot of hanging time giving the type of each kick', fontsize= 16)
ax3.set_ylabel('')
fig.suptitle('PFFScoutingData Summary', fontsize = 22, y=.94);

<a id='trk'></a>
## Now we explore the tracking datasets
### We start by doing some aggregation

In [ ]:
try:
    tracking2018.groupby(['displayName','gameId','playId']).agg({'dis' : 'sum', 's':'mean','team':st.mode}).reset_index()
except:
    print(colored('Guess what?!, ...It happened!',color = 'red'))

<center><img src="https://c.tenor.com/FkEo2jYLDrYAAAAd/what-a-coincidence-reassure.gif"></center>

In [ ]:
t_df = tracking2018.groupby(['displayName','gameId','playId']).agg({'dis' : 'sum', 's':'mean','team':lambda x: [i for i in x]}).reset_index().copy()

#### Investigating this (don't judge me, I just need to know )

In [ ]:
for n, i in enumerate(t_df.team.values):
    for j in i:
        if j != i[0]:
            print(n)
            break

In [ ]:
t_df.iloc[106540,:]

In [ ]:
tracking2018[(tracking2018.gameId==2018093011) & (tracking2018.playId == 3604) & (tracking2018.displayName == 'Michael Thomas')].sample(10)

#### Told ya, it happens!! .. two `Michael Thomas` same game same play, different teams! 
<center><img src="https://c.tenor.com/QXVs4QWLlzkAAAAC/spider-man.gif"  width='500' height="500"></center>

### Now We fix this problem and make an aggregation for the three seasons by each player and each play

In [ ]:
del t_df
tracking18_agg = tracking2018.groupby(['nflId','displayName', 'jerseyNumber', 'position', 'team','gameId','playId']).agg({'time':'min', 'dis' : 'sum', 's':['mean','max','min'],'a':['mean','max','min']}).copy()
tracking19_agg = tracking2019.groupby(['nflId','displayName', 'jerseyNumber', 'position', 'team','gameId','playId']).agg({'time':'min', 'dis' : 'sum', 's':['mean','max','min'],'a':['mean','max','min']}).copy()
tracking20_agg = tracking2020.groupby(['nflId','displayName', 'jerseyNumber', 'position', 'team','gameId','playId']).agg({'time':'min', 'dis' : 'sum', 's':['mean','max','min'],'a':['mean','max','min']}).copy()
tracking18_agg.columns = ['beg_time', 'dis_sum', 's_mean', 's_max', 's_min', 'a_mean', 'a_max', 'a_min']
tracking19_agg.columns = ['beg_time', 'dis_sum', 's_mean', 's_max', 's_min', 'a_mean', 'a_max', 'a_min']
tracking20_agg.columns = ['beg_time', 'dis_sum', 's_mean', 's_max', 's_min', 'a_mean', 'a_max', 'a_min']
tracking18_agg.reset_index(inplace= True)
tracking19_agg.reset_index(inplace= True)
tracking20_agg.reset_index(inplace= True)
tracking18_agg = tracking18_agg.merge(games, how= 'inner', left_on= 'gameId',right_on='gameId').merge(plays, left_on= ['gameId','playId'], right_on = ['gameId','playId']).merge(players, left_on= ['nflId'], right_on=['nflId']).copy()
tracking19_agg = tracking19_agg.merge(games, how= 'inner', left_on= 'gameId',right_on='gameId').merge(plays, left_on= ['gameId','playId'], right_on = ['gameId','playId']).merge(players, left_on= ['nflId'], right_on=['nflId']).copy()
tracking20_agg = tracking20_agg.merge(games, how= 'inner', left_on= 'gameId',right_on='gameId').merge(plays, left_on= ['gameId','playId'], right_on = ['gameId','playId']).merge(players, left_on= ['nflId'], right_on=['nflId']).copy()
gc.collect()
tracking18_agg.sample(4)

### checking again for duplicated names problem

In [ ]:
tracking18_agg[(tracking18_agg.playId == 3604) & (tracking18_agg.displayName_x == 'Michael Thomas')]

<center><img src="https://c.tenor.com/790jxHUkKLIAAAAC/loser-joey.gif" width='500' height="500"></center>

### Merging it all together

In [ ]:
tracking_agg = pd.concat([tracking18_agg, tracking19_agg, tracking20_agg], ignore_index=True, sort=False)
tracking_agg['team_name'] = np.select([tracking_agg.team =='home' ,tracking_agg.team =='away'], [tracking_agg.homeTeamAbbr, tracking_agg.visitorTeamAbbr])
tracking_agg['side'] = np.where(tracking_agg.possessionTeam == tracking_agg.team_name, 'offense','defense')
tracking_agg['team_conference'] = tracking_agg.team_name.apply(lambda x: team_abb[x][-2])
tracking_agg['team_region'] = tracking_agg.team_name.apply(lambda x: team_abb[x][-1])
tracking_agg['team_division'] = tracking_agg.team_conference + ' ' + tracking_agg.team_region
tracking_agg.displayName_x = tracking_agg.displayName_x.str.lower().copy()
tracking_agg.displayName_y = tracking_agg.displayName_y.str.lower().copy()
tracking_agg.playDescription = tracking_agg.playDescription.str.lower().copy()

tracking_agg['displayName_x_abb'] = tracking_agg.displayName_x.apply(lambda nm : str(nm)[0]+'.')
tracking_agg['displayName_x_abb'] = tracking_agg.displayName_x_abb + tracking_agg.displayName_x.str.extract('\s(\w+)').loc[:,0]
tracking_agg['displayName_y_abb'] = tracking_agg.displayName_y.apply(lambda nm : str(nm)[0]+'.')
tracking_agg['displayName_y_abb'] = tracking_agg.displayName_y_abb + tracking_agg.displayName_y.str.extract('\s(\w+)').loc[:,0]
del tracking18_agg, tracking19_agg, tracking20_agg
l = []
for name_x_ab, name_y_ab, name_x, name_y, playdes in zip(tracking_agg.displayName_x_abb,
                                   tracking_agg.displayName_y_abb,
                                   tracking_agg.displayName_x,
                                   tracking_agg.displayName_y,
                                   tracking_agg.playDescription):
    if (name_x_ab in playdes) or (name_y_ab in playdes) or(name_x in playdes) or (name_y in playdes):
        l.append('key player')
    else:
        l.append('non key player')
tracking_agg['player_is_key'] = l
gc.collect()
tracking_agg.head()


<center><img src="https://c.tenor.com/Oc4nf8N08jIAAAAd/mind-blow-galaxy.gif"  width='500' height="500"></center>

### Clarification
#### `side` feature added above is meant whether the player in on the team with the ball or not as offense mean that player is with the team that have the possession and defense if not (and not to be confused with fotball offense adn defense teams)

## Distance covered and speed
### In this part will be look into the distance covered and speed of players 

### First we make a KDE plot of average distance ran by a player

In [ ]:
plt.figure(figsize=(10,6))
sns.kdeplot(data=tracking_agg, x='dis_sum', hue= 'season', fill=False, palette='viridis',linewidth=2)
plt.title('Average distance ran by a player in a play', fontsize=18);

> ### Distance covered by a players seems to vary a lot

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(y= 's_mean', x= 'dis_sum', data= tracking_agg,s=100, palette= 'viridis',hue='position', alpha=.7)
plt.title('Relation between speed and distance made by each player in a play',fontsize=16)
plt.legend(loc='lower right',ncol=2,title='Position',title_fontsize=14);

In [ ]:
fig, ((ax11, ax12, ax13, ax14),(ax21, ax22, ax23, ax24)) = plt.subplots(2,4, figsize=(24,10))
plt.subplots_adjust(hspace=.25)
sns.barplot(x= 'season', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='viridis', hue='side',alpha=.9,ax=ax11)
ax11.set_title('Side in the play',fontsize= 16)
ax11.set_xticks([-1,0,1,2,3])
ax11.set_xticklabels(['',2018, 2019, 2020,''])
ax11.tick_params(labelsize=14)
ax11.set_ylabel('Distance (yards)',fontsize=14)
ax11.set_xlabel('')
ax11.legend(ncol=1,fontsize=10,title='Side',title_fontsize= 12,loc='lower right');

sns.barplot(x= 'season', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='viridis', hue='team', alpha=.9,ax=ax12)
ax12.set_title('Player team',fontsize=16)
ax12.set_xticks([-1,0,1,2,3])
ax12.set_xticklabels(['',2018, 2019, 2020,''])
ax12.tick_params(labelsize=14)
ax12.set_ylabel('')
ax12.set_xlabel('')
ax12.legend(ncol=1,fontsize=10,title='Team',title_fontsize= 12,loc='lower right');

sns.barplot(x= 'season', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='viridis', hue='quarter',alpha=.9,ax=ax13)
ax13.set_title('Quarter',fontsize=16)
ax13.set_xticks([-1,0,1,2,3])
ax13.set_xticklabels(['',2018, 2019, 2020,''])
ax13.tick_params(labelsize=14)
ax13.set_ylabel('')
ax13.set_xlabel('')
ax13.legend(ncol=1,fontsize=10,title='Quarter',title_fontsize= 12,loc='lower right');

sns.barplot(x= 'season', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='viridis', hue='specialTeamsPlayType',alpha=.9,ax=ax14)
ax14.set_title('Play type',fontsize=16)
ax14.set_xticks([-1,0,1,2,3])
ax14.set_xticklabels(['',2018, 2019, 2020,''])
ax14.tick_params(labelsize=14)
ax14.set_ylabel('')
ax14.set_xlabel('')
ax14.legend(ncol=1,fontsize=10,title='Play type',title_fontsize= 12,loc='center left');

fig.text(.5,.92, 'Avg distance ran in a play by a player',horizontalalignment='center', fontsize=18)

sns.barplot(x= 'season', y= 's_mean', data= tracking_agg, ci=99.9, palette='viridis', hue='side',alpha=.9,ax=ax21)
ax21.set_title('')
ax21.set_xticks([-1,0,1,2,3])
ax21.set_xticklabels(['',2018, 2019, 2020,''])
ax21.tick_params(labelsize=14)
ax21.set_ylabel('Speed (y/s)',fontsize=14)
ax21.set_xlabel('')
ax21.legend(ncol=1,fontsize=10,title='Side',title_fontsize= 12,loc='lower right');

sns.barplot(x= 'season', y= 's_mean', data= tracking_agg, ci=99.9, palette='viridis', hue='team', alpha=.9, ax=ax22)
ax22.set_title('')
ax22.set_xticks([-1,0,1,2,3])
ax22.set_xticklabels(['',2018, 2019, 2020,''])
ax22.tick_params(labelsize=14)
ax22.set_ylabel('')
ax22.set_xlabel('')
ax22.legend(ncol=1,fontsize=10,title='Team',title_fontsize= 12,loc='lower right')

sns.barplot(x= 'season', y= 's_mean', data= tracking_agg, ci=99.9, palette='viridis', hue='quarter',alpha=.9,ax=ax23)
ax23.set_title('')
ax23.set_xticks([-1,0,1,2,3])
ax23.set_xticklabels(['',2018, 2019, 2020,''])
ax23.tick_params(labelsize=14)
ax23.set_ylabel('')
ax23.set_xlabel('')
ax23.legend(ncol=1,fontsize=10,title='Quarter',title_fontsize= 12,loc='lower right');

sns.barplot(x= 'season', y= 's_mean', data= tracking_agg, ci=99.9, palette='viridis', hue='specialTeamsPlayType', alpha=.9, ax=ax24)
ax24.set_title('')
ax24.set_xticks([-1,0,1,2,3])
ax24.set_xticklabels(['',2018, 2019, 2020,''])
ax24.tick_params(labelsize=14)
ax24.set_ylabel('')
ax24.set_xlabel('')
ax24.legend(ncol=1,fontsize=10,title='Play type',title_fontsize= 12,loc='center left')

fig.text(.5,.48, 'Avg spead of a player in a play',horizontalalignment='center', fontsize=18)


fig.suptitle('Distance and Speed Summary', fontsize=20,y=.98);

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(24,11))
sns.barplot(x= 'position', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='viridis', hue='season',ax=ax1,
            order= tracking_agg.groupby('position')['dis_sum'].mean().sort_values(ascending= False).index)
ax1.set_title('Avg of distance ran in a play given the position and sesaon',fontsize=20)
ax1.tick_params(labelsize=14)
ax1.set_ylabel('Avg Distance ran (yards)',fontsize=16)
ax1.set_xlabel('')
ax1.legend(ncol=1,fontsize=14,title='Season',title_fontsize= 14);

sns.barplot(x= 'position', y= 's_mean', data= tracking_agg, ci=99.9, palette='viridis', hue='season',ax=ax2,
            order= tracking_agg.groupby('position')['s_mean'].mean().sort_values(ascending= False).index)
ax2.set_title('Avg speed in a play given the position and sesaon',fontsize=20)
ax2.tick_params(labelsize=14)
ax2.set_ylabel('Avg speed (y/s)',fontsize=16)
ax2.set_xlabel('')
ax2.legend(ncol=1,fontsize=14,title='Season',title_fontsize= 14);

### Using box plot to explore outliers

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(24,11))
sns.boxplot(x= 'position', y= 'dis_sum', data= tracking_agg, palette='viridis', hue='season',
            order= tracking_agg.groupby('position')['dis_sum'].mean().sort_values(ascending= False).index,ax=ax1)
ax1.set_title('Distance ran in a play given the position and sesaon',fontsize=20)
ax1.tick_params(labelsize=14)
ax1.set_ylabel('Distance ran (yards)',fontsize=16)
ax1.set_xlabel('')
ax1.legend(ncol=1,fontsize=12,title='Season',title_fontsize= 14);

sns.boxplot(x= 'position', y= 's_mean', data= tracking_agg, palette='viridis', hue='season',
            order= tracking_agg.groupby('position')['s_mean'].mean().sort_values(ascending= False).index,ax=ax2)
ax2.set_title('Speed average in a play given the position and sesaon',fontsize=20)
ax2.tick_params(labelsize=14)
ax2.set_ylabel('Speed (y/s)',fontsize=16)
ax2.set_xlabel('')
ax2.legend(ncol=1,fontsize=12,title='Season',title_fontsize= 14);

> ### Seems like there are a lot of outliers

### Now we plot the same variables with only changing the season variable to quarters

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(28,11))
sns.boxplot(x= 'position', y= 'dis_sum', data= tracking_agg, palette='viridis', hue='quarter',
            order= tracking_agg.groupby('position')['dis_sum'].mean().sort_values(ascending= False).index,ax=ax1)
ax1.set_title('Distance ran in a play given the position and quarter',fontsize=20)
ax1.tick_params(labelsize=14)
ax1.set_ylabel('Distance ran (yards)',fontsize=16)
ax1.set_xlabel('')
ax1.legend(ncol=1,fontsize=12,title='quarter',title_fontsize= 14);

sns.boxplot(x= 'position', y= 's_mean', data= tracking_agg, palette='viridis', hue='quarter',
            order= tracking_agg.groupby('position')['s_mean'].mean().sort_values(ascending= False).index,ax=ax2)
ax2.set_title('Speed average in a play given the position and quarter',fontsize=20)
ax2.tick_params(labelsize=14)
ax2.set_ylabel('Speed (y/s)',fontsize=16)
ax2.set_xlabel('')
ax2.legend(ncol=1,fontsize=12,title='quarter',title_fontsize= 14);

### Next we check teams statistics
#### We will order the teams to focus on the last Super Bowl and Conference winners

In [ ]:
NE = requests.get("https://cdn.freebiesupply.com/images/large/2x/new-england-patriots-logo-transparent.png")
NE = Image.open(BytesIO(NE.content))
KC = requests.get("https://logos-world.net/wp-content/uploads/2020/05/Kansas-City-Chiefs-logo.png")
KC = Image.open(BytesIO(KC.content))
TB = requests.get("https://cdn.freebiesupply.com/images/large/2x/tampa-bay-buccaneers-logo-transparent.png")
TB = Image.open(BytesIO(TB.content))
SF = requests.get("https://assets.stickpng.com/images/580b585b2edbce24c47b2b55.png")
SF = Image.open(BytesIO(SF.content))
LA = requests.get("https://logodownload.org/wp-content/uploads/2020/09/la-rams-logo-0.png")
LA = Image.open(BytesIO(LA.content))
finalists = [TB,KC,SF,NE,LA]
for i in finalists:
    i.thumbnail((250,250))
    LA.convert('RGBA')
img = Image.new('RGBA', (1250, 250), color = (255, 255, 255))
x = 0
for i,j in enumerate(finalists):
    if i not in [0,4]:
        img.paste(j,box=(x,50))
    else:
        img.paste(j, box= (x,0))
    x += 250
fig, axeses = plt.subplots(1,5, figsize=(27,7))

for ax, logo in zip(axeses, finalists):
    ax.imshow(logo,interpolation='nearest')
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_facecolor('white')
fig.suptitle("Finalists of the last three seasons\n\n\n",y=.95, fontsize=20)

axeses[0].set_title('NFC Champion 2020\nWon Super Bowl\n', fontsize=16)
axeses[1].set_title('AFC Champion 2019/2020\nWon Super Bowl 2019 lost 2020\n\n\n', fontsize=16)
axeses[2].set_title('NFC Champion 2019\nLost Super Bowl\n\n\n', fontsize=16)
axeses[3].set_title('AFC Champion 2018\nWon Super Bowl\n\n\n', fontsize=16)
axeses[4].set_title('\n\nNFC Champion 2018\nLost Super Bowl', fontsize=16);

### Making an order list for teams and also divisions

In [ ]:
team_ord = ['TB','KC','SF','NE','LA','IND','WAS','MIA',
            'NYG','DET','NO','NYJ','CLE','MIN','HOU','TEN',
            'CIN','GB','BUF','DEN','JAX','DAL','ARI','LAC',
            'CHI','SEA','PIT','CAR','ATL', 'BAL','OAK','LV']
div_ord = ['AFC south','NFC south','AFC east', 'NFC east',
           'AFC north', 'NFC north', 'AFC west', 'NFC west']

### Let's explore distance and speed

In [ ]:
fig = plt.figure(figsize=(28,11))
grid =  gridspec.GridSpec(2,6 ,figure=fig, hspace=.25,wspace=.15 )
ax1 = fig.add_subplot(grid[0, 0])
ax12 = fig.add_subplot(grid[0, 1:3])
ax13 = fig.add_subplot(grid[0, 3:])
ax2 = fig.add_subplot(grid[1, 0])
ax22 = fig.add_subplot(grid[1, 1:3])
ax23 = fig.add_subplot(grid[1, 3:])

sns.barplot(x= 'team_conference', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='Blues', hue='season', ax= ax1)
ax1.set_title('Avg Distance ran in a play by a player\ngiven team conference and sesaon',fontsize=12)
ax1.tick_params(labelsize=12)
ax1.set_ylabel('Distance (yards)',fontsize=14)
ax1.set_xlabel('')
ax1.set_ylim(0,37)
ax1.get_legend().remove()

sns.barplot(x= 'team_region', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='Blues', hue='season', ax= ax12)
ax12.set_title('Avg Distance ran in a play by a player\ngiven team region and sesaon',fontsize=12)
ax12.tick_params(labelsize=12)
# ax12.set_yticks([])
ax12.set_ylabel('')
ax12.set_xlabel('')
ax12.set_ylim(0,37)
ax12.get_legend().remove()

sns.barplot(x= 'team_division', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='Blues', hue='season', ax= ax13)
ax13.set_title('Avg Distance ran in a play by a player\ngiven his division and sesaon',fontsize=12)
ax13.tick_params(labelsize=12)
# ax13.set_yticks([])
ax13.set_ylabel('')
ax13.set_xlabel('')
ax13.set_ylim(0,37)
ax13.get_legend().remove()


sns.barplot(x= 'team_conference', y= 's_mean', data= tracking_agg, ci=99.9, palette='Blues', hue='season', ax= ax2)
ax2.set_title('Avg speed of a player\ngiven conference and sesaon',fontsize=12)
ax2.tick_params(labelsize=12)
ax2.set_ylabel('Speed (y/s)',fontsize=14)
ax2.set_xlabel('')
ax2.set_ylim(0,4.5)
ax2.get_legend().remove()

sns.barplot(x= 'team_region', y= 's_mean', data= tracking_agg, ci=99.9, palette='Blues', hue='season', ax= ax22)
ax22.set_title('Avg speed of a player\ngiven his team region and sesaon',fontsize=12)
ax22.tick_params(labelsize=12)
# ax22.set_yticks([])
ax22.set_ylabel('')
ax22.set_xlabel('')
ax22.set_ylim(0,4.5)
ax22.get_legend().remove()

sns.barplot(x= 'team_division', y= 's_mean', data= tracking_agg, ci=99.9, palette='Blues', hue='season', ax= ax23)
ax23.set_title('Avg speed of a player\ngiven his division and sesaon',fontsize=12)
ax23.tick_params(labelsize=12)
# ax23.set_yticks([])
ax23.set_ylabel('')
ax23.set_xlabel('')
ax23.set_ylim(0,4.5)
ax23.get_legend().remove()
h, l = ax1.get_legend_handles_labels()
fig.legend(h,l, ncol=3, fontsize=10,title='Season',title_fontsize= 12,bbox_to_anchor=(.57,.95))
fig.suptitle('Distance and speed of player in each play', fontsize=20);

### Now we explore by teams

In [ ]:
fig, [ax1,ax2,ax3] = plt.subplots(3,1, figsize=(25,16))

sns.barplot(x= 'team_name', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='Blues', hue='season',order= team_ord, ax= ax1)
ax1.set_title('Avg Distance ran in a play by a player given his team and the sesaon',fontsize=16)
ax1.tick_params(labelsize=12)
ax1.set_ylabel('Distance (yards)',fontsize=14)
ax1.set_xlabel('')
ax1.set_ylim(15,37)
ax1.get_legend().remove()
sns.barplot(x= 'homeTeamAbbr', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='Blues', hue='season', order= team_ord, ax= ax2)
ax2.set_title('Avg Distance ran in a play by a player at home stadium given the team and the sesaon',fontsize=16)
ax2.tick_params(labelsize=12)
ax2.set_ylabel('Distance (yards)',fontsize=14)
ax2.set_xlabel('')
ax2.set_ylim(15,37)
ax2.get_legend().remove()

sns.barplot(x= 'visitorTeamAbbr', y= 'dis_sum', data= tracking_agg, ci=99.9, palette='Blues', hue='season',order= team_ord, ax= ax3)
ax3.set_title('Avg Distance ran in a play by a player playing away given the team and the sesaon',fontsize=16)
ax3.tick_params(labelsize=12)
ax3.set_ylabel('Distance (yards)',fontsize=14)
ax3.set_xlabel('')
ax3.set_ylim(15,37)
ax3.get_legend().remove()

fig.suptitle('Avg Distance covered summary by teams', y= .94, fontsize=20)
h, l = ax1.get_legend_handles_labels()
fig.legend(h,l, ncol=3, fontsize=14,title='Season',title_fontsize= 16,bbox_to_anchor=(.29,.93));

In [ ]:
plt.figure(figsize=(24,5))
sns.boxplot(x= 'position', y= 's_mean', data= tracking_agg, palette='viridis', hue='season',
            order= tracking_agg.groupby('position')['s_mean'].mean().sort_values(ascending= False).index)
plt.title('Speed average in a play given the position and sesaon',fontsize=22)
plt.xticks(fontsize=16)
plt.yticks(fontsize=14)
plt.ylabel('Speed (y/s)',fontsize=16)
plt.xlabel('')
plt.legend(ncol=1,fontsize=12,title='Season',title_fontsize= 14);

In [ ]:
plt.figure(figsize=(18,6))
sns.countplot(data= tracking_agg, x= 'Position', hue= 'player_is_key', palette= 'viridis');

In [ ]:
tracking_agg.player_is_key.value_counts()

<a id='PMEDA'></a>


<center><h1>Plotting Player Movement</h1></center>
<h1></h1>
<center><img src="https://www.reactiongifs.us/wp-content/uploads/2017/10/Time-outNFL.gif" width='600' height="600"></center>

### Now let's take a time out from data analysis and focus a little on coding,
### please note that in this part functional programming will be heavily used to come out with nice tools (functions) to plot tracking data without a lot of code repetion
#### P.S most of code in this section is hidden for Aesthetic reasons (too many lines) so you can unhide it to show the code of each function.

### First we make a function called `football_field` to plot a football field

In [ ]:
def football_field(ax):
    """
    this function set the plot to look like a football field
    input:
    ax: an axes
    return
    matplotlib axes with football field drawn
    """
    sns.set_style('ticks')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.grid(axis='x',zorder=1)
    ax.xaxis.grid(linestyle='dotted',linewidth=2.5,color='white',zorder=1)
    ax.set_facecolor('limegreen')
    ax.set_xlim(0,120)
    ax.set_ylim(0,53.4)
    
    ax.axvspan(0, 10,linestyle='', color='white', alpha=1, zorder=2)
    ax.axvspan(110,120, linestyle='', color='white', alpha=1, zorder=2)
    ax.axvspan(0, 10, linestyle='', color='fuchsia', alpha=.4, zorder=3)
    ax.axvspan(110, 120, linestyle='', color='fuchsia', alpha=.4, zorder=3)
    
    ax.xaxis.set_major_locator(MultipleLocator(10))
    ax.xaxis.set_major_formatter('{x:.0f}')
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    ax.axvline(10, color='white',linewidth=4 ,zorder=3)
    ax.axvline(110, color='white',linewidth= 4,zorder= 3)
    ax.axvline(60, color='white', linewidth=4,zorder=3)
    for i in range(11,110,2):
        ax.axvline(i,.48,.52,linewidth= 2 ,zorder=4, alpha=.9,color='white')
    
    ax.tick_params(axis='x', which='minor',direction ='in', top=True,bottom=True, length = 8,width= 1.5,color='white', zorder=3)
    ax.tick_params(axis='x', which='major',direction ='in', top=True,labeltop= True,bottom=True, length = 10,width= 2.5,pad=-35,labelsize='small', color='white',labelcolor='white',zorder=3)
    ax.tick_params(axis='y', which='major', left=False)
    ax.set_xticks([i for i in range(0,121,10)])
    ax.set_xticklabels(['','','1 0','2 0','3 0','4 0','5 0','4 0','3 0','2 0','1 0','',''],fontsize=22,fontfamily='serif',fontweight='heavy')
    ax.set_yticks([])
    return ax

### Testing the function

In [ ]:
fig, ax = plt.subplots(figsize=(12,5.5))
ax  = football_field(ax)

<center><h3>Looking Good!</h3></center>

<center><img src="https://www.reactiongifs.us/wp-content/uploads/2018/11/2018-NFL-Football-1.gif" width=600 height=500></center>

### The coming function is called `get_player_data` and is meant to extract players' data

In [ ]:
def get_player_data(year, playername=None, nflid=None):
    '''
    this function get player data
    input:
    playername: string, the name of the player
    nflid: int, unique identifier of the player
    year int, the year which its data is being plotted
    return:
    player weight, height and age
    '''
    if playername == nflid == None:
        print(colored('kindly enter either player name or his NFL ID','red'))
        return None
    if playername != None:
        if playername not in players.displayName.values:
            print(colored('Wrong name!, {} is not in players database you can try this with NFL ID'.format(playername),'red'))
            return None
        if playername in missing_players.keys():
            weight = missing_players[playername][1]
            height = missing_players[playername][0]
            Age = '--'
        elif playername in duplicated_names:
            nflid = int(input('this name is duplicated kindly enter the nflid of the player'))
            weight = players[players.nflId == nflid].weight.values[0]
            height = players[players.nflId == nflid].highcm.values[0]
            Age = players[players.nflId == nflid].age.values[0] + (year - 2018)
        else:
            weight = players[players.displayName == playername].weight.values[0]
            height = players[players.displayName == playername].highcm.values[0]
            Age = players[players.displayName == playername].age.values[0] + (year - 2018)
        return weight, height, Age
    if nflid != None:
        if nflid not in players.nflId.values:
            print(colored('Wrong NFL ID {} is not in players database, please make sure you entered the correct ID','red'))
            return None
        else:
            weight = players[players.nflId == nflid].weight.values[0]
            height = players[players.nflId == nflid].highcm.values[0]
            Age = players[players.nflId == nflid].age.values[0] + (year - 2018)
        return weight, height, Age

### Now we make a function called `get_game_id` to get game id

In [ ]:
games.set_index('gameId',inplace=True)

In [ ]:
def get_game_id(season, week, hometeam= None, awayteam= None, playername= None):
    '''
    this function gets the unique id of a game
    inputs:
    year: int, the year in which the game was in
    home/away team: strng, the home/away team in this game
    playername: string, player name that was in this game (might need to enter unique nflId of the player)
    return:
    gameid: int, unique game id
    '''
    nflid = 0
    try:
        if hometeam == awayteam == playername == None:
            print(colored('Couldn\'t get game id!, please make sure that you enter at least one of these args [hometeam, awayteam, playername]','red'))
            return None
    except:
        pass
    
    if season not in [2018,2019,2020]:
        print(colored('wrong season!, please make sure that selected season is in [2018:2020]','red'))
        return None
    elif ((week > 17) or (week < 1)) or type(week) != int:
        print(colored('wrong week value!, please make sure that selected week is in [1:17]','red'))
        return None
    try:
        if playername in duplicated_names:
            nflid = int(input('player name is duplicated kindly enter the unique NFL id of the player\n'))
            nflid = players[players.displayName == playername].nflId.values[0]
    except:
        print(colored('{} doesn\'t have tracking data in {}!'.format(playername, season),'red'))
        return None

    if hometeam != None:
        try:
            gid = games[(games.season == season) & (games.week == week) & (games.homeTeamAbbr == hometeam)].index[0] 
            return gid
        except:
            print(colored('make sure you selected the right home team!','red'))
    
    elif awayteam != None:
        try:
            gid = games[(games.season == season) & (games.week == week) & (games.visitorTeamAbbr == awayteam)].index[0]
            return gid
        except:
            print(colored('make sure you selected the right visitor team!','red'))
    else:
        if season == 2018:
            try:
                games_this_week =list(games[(games.season == season) & (games.week == week)].index)
                if nflid != 0:
                    gid = tracking2018[(tracking2018.nflId == nflid) & (tracking2018.gameId.isin(games_this_week))].gameId.values[0]
                else:
                    gid = tracking2018[(tracking2018.displayName == playername) & (tracking2018.gameId.isin(games_this_week))].gameId.values[0]
                return gid
            except:
                 print(colored('{} doesn\'t have tracking data in {}!'.format(playername, season),'red'))
        elif season == 2019:
            try:
                games_this_week =list(games[(games.season == season) & (games.week == week)].index)
                if nflid != 0:
                    gid = tracking2019[(tracking2019.nflId == nflid) & (tracking2019.gameId.isin(games_this_week))].gameId.values[0]
                else:
                    gid = tracking2019[(tracking2019.displayName == playername) & (tracking2019.gameId.isin(games_this_week))].gameId.values[0]
                return gid
            except:
                 print(colored('{} doesn\'t have tracking data in {}!'.format(playername, season),'red'))
        elif season == 2020:
            try:
                games_this_week =list(games[(games.season == season) & (games.week == week)].index)
                if nflid != 0:
                    gid = tracking2020[(tracking2020.nflId == nflid) & (tracking2020.gameId.isin(games_this_week))].gameId.values[0]
                else:
                    gid = tracking2020[(tracking2020.displayName == playername) & (tracking2020.gameId.isin(games_this_week))].gameId.values[0]
                return gid
            except:
                 print(colored('{} doesn\'t have tracking data in {}!'.format(playername, season),'red'))

### Now we make some functions to get specific games' id's to that end we will make these functions:
* `get_team_games`
* `get_player_games`

In [ ]:
def get_team_games(team, season=0, where= 'all'):
    '''
    this function returns all games id's for a team
    team: string, team name for which games id's are being extracted
    season: int, the season on which games were played if let it will exrtact all games in all season in the database
    where: string, extract games only playing in home, away or both
    return:
    team_games: list containt games id's
    '''
    if team not in team_abb.keys():
        print(colored('Wrong team name! cold\'nt find values for `{}`'.format(team),'red'))
        return None
    
    if where not in ['home', 'away', 'all']:
        print(colored('Wrong value for `where` arg!, please make sure it\'s within [home, away, all]','red'))
        return None
    
    elif season == 0:
        if where == 'all':
            team_games = list(games[(games.homeTeamAbbr == team) | (games.visitorTeamAbbr == team)].index)
        elif where == 'home':
            team_games = list(games[games.homeTeamAbbr == team].index)
        elif where == 'away':
            team_games = list(games[games.visitorTeamAbbr == team].index)
        return team_games
    
    elif season not in [2018, 2019, 2020]:
        print(colored('Wrong season! please make sure that season is in [2018:2020] period','red'))
        return None
    
    else:
        if where == 'all':
            team_games = list(games[(games.season == season) & ((games.homeTeamAbbr == team) | (games.visitorTeamAbbr == team))].index)
        elif where == 'home':
            team_games = list(games[(games.season == season) & (games.homeTeamAbbr == team)].index)
        elif where == 'away':
            team_games = list(games[(games.season == season) & (games.visitorTeamAbbr == team)].index)      
        return team_games
    

In [ ]:
def get_player_games(season, playername= None, nflid= None, where= 'all'):
    '''
    this function returns all games id's for a players
    playername: string, player name for which games id's are being extracted
    season: int, the season on which games were played if let it will exrtact all games in all season in the database
    where: string, extract games only playing in home, away or both
    return:
    team_games: list containt games id's
    '''
    if playername == nflid == None:
        print(colored('kindly enter either player name or his NFL ID','red'))
        return None
    if where not in ['home', 'away', 'all']:
        print(colored('Wrong value for `where` arg!, please make sure it\'s within [home, away, all]','red'))
        return None
    tracking = eval('tracking'+str(season))
    if playername != None:
        if playername in duplicated_names:
            nflid = int(input('this name is duplicated kindly enter the nflid of the player'))
            if where == 'all':
                player_games = list(tracking[tracking.nflId == nflid].gameId.unique())
            elif where == 'home':
                player_games = list(tracking[(tracking.nflId == nflid) & (tracking.team == 'home')].gameId.unique())
            elif where =='away':
                player_games = list(tracking[(tracking.nflId == nflid) & (tracking.team == 'away')].gameId.unique())
        else:
            if where == 'all':
                player_games = list(tracking[tracking.displayName == playername].gameId.unique())
            elif where == 'home':
                player_games = list(tracking[(tracking.displayName == playername) & (tracking.team == 'home')].gameId.unique())
            elif where =='away':
                player_games = list(tracking[(tracking.displayName == playername) & (tracking.team == 'away')].gameId.unique())
        
    elif nflid != None:
        if nflid not in players.nflId.values:
            print(colored('Wrong NFL ID {} is not in players database, please make sure you entered the correct ID','red'))
            return None
        else:
            if where == 'all':
                player_games = list(tracking[tracking.nflId == nflid].gameId.unique())
            elif where == 'home':
                player_games = list(tracking[(tracking.nflId == nflid) & (tracking.team == 'home')].gameId.unique())
            elif where =='away':
                player_games = list(tracking[(tracking.nflId == nflid) & (tracking.team == 'away')].gameId.unique())
    del tracking
    return player_games

### This coming function is called `get_plays` and is meant to get plays within a game

In [ ]:
def get_plays(gameid, posteam= None):
    '''
    this function print all plays within a game
    inputs:
    gameid: int, unique identifier of a game
    posteam: string, optional, name of the team that possesses the ball
    return:
    list of all plays within a game
    '''
    if posteam == None:
        plays_list = list(plays[plays.gameId == gameid].playId)
        if len(plays_list) == 0:
            print(colored('make sure that you insert the correct values','red'))
            return
        else:
            return plays_list
    else:
        plays_list = list(plays[(plays.gameId == gameid) & (plays.possessionTeam == posteam)].playId)
        if len(plays_list) == 0:
            print(colored('Make sure that you insert the correct values!','red'))
            return
        else:
            return plays_list

### As the name suggest `get_play_data` is meant to extract data about a play

In [ ]:
def get_play_data(df, gameid, playid):
    '''
    this function gets a single play data
    input:
    df: dataFrame, sub tracking dataFrame to extract data from
    gameid int, unique identifier of a game
    return
    week, date, home, away, posteam, team, direction, gtype, and result of the play
    '''
    week = games.loc[gameid,'week']
    date = games.loc[gameid,'gameDate']
    home = games.loc[gameid,'homeTeamAbbr']
    away = games.loc[gameid,'visitorTeamAbbr']
    posteam = plays[(plays.gameId == gameid) & (plays.playId == playid)].possessionTeam.values[0]
    team = df['team'].values[0]
    posteam = plays[(plays.gameId == gameid) & (plays.playId == playid)].possessionTeam.values[0]
    direction = df['playDirection'].values[0]
    gtype = plays[(plays.gameId == gameid) & (plays.playId == playid)].specialTeamsPlayType.values[0]
    result = plays[(plays.gameId == gameid) & (plays.playId == playid)].specialTeamsResult.values[0]
    return week, date, home, away, posteam, team, direction, gtype, result

### The next function is called `plot_custom` and it use past functions to comout with a plot 

In [ ]:
def plot_custom(df, playername, year, gameid=None, playid=None, ball= False):
    """
    this is the main plotting func, it plots player movmement and heat map of that movment
    inputs:
    df: sub-dataframe to plot from, extracted from tracking2018
    playrname: string, player we are try to plot his movement
    year: int, year in whic tracking is in
    ball: bool, boolean to plot the ball 
    return:
    figure of player movement with direction, ball movment (if plotting a single play) and also heat map of player movment
    """
    sns.set_style('ticks')
    if ball == True:
        g_ball_df = eval('ball_'+str(year)[-2:]+'_df')
        if (gameid != None) and (playid == None):
            g_ball_df = g_ball_df[g_ball_df.gameId == gameid]
        elif (gameid != None) and (playid != None):
            g_ball_df = g_ball_df[(g_ball_df.gameId == gameid) & (g_ball_df.playId == playid)]
    fig = plt.figure(figsize=(1.2*25,1.2*5.5))
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=.02,hspace=.2)
    ax = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)
    ax = football_field(ax)
    ax2 = football_field(ax2)
    sns.kdeplot(x='x', y='y', data= df, cmap= hcmap,fill=True,alpha=.85,ax=ax,zorder=5)
    ax2.quiver(df.x, df.y, df.x, df.y,angles= (90- df.dir ), zorder=5, pivot="mid",edgecolor='none',color='blue',alpha=.3,headwidth = 7 )
    if ball == True:
        ax2.scatter(g_ball_df.x, g_ball_df.y ,color='brown', alpha=.7, marker="o", s= 80, label='Ball', zorder=6)
        ax2.legend(loc='upper right')
    weight, height, Age = get_player_data(year,playername)
    fig.suptitle('Plotting {} Movement\nAge:{} yrs | Weight: {} lb | Height: {} cm'.format(playername,Age,weight,height),fontsize=16)
    ax.set_title('Heat map of {} movment'.format(playername),fontsize=16)
    ax2.set_title('Movement and direction of {}'.format(playername),fontsize=16)
    if playid != None:
        p_week, p_date, p_home, p_away, p_posteam, p_team, p_direction, p_gtype, p_result = get_play_data(df, gameid= gameid, playid= playid)
        boxstr = 'Week:{}\nOn: {}\nHome: {}\nAway: {}\nPlayer is in {}\n{}\'s ball\nType: {}\nResult: {}'.format(p_week, p_date,
                                                                                                                 p_home , p_away, p_team,
                                                                                                                 p_posteam, p_gtype, p_result)
        fig.text(.49,.45,boxstr, fontsize=12, bbox=dict(boxstyle="round",ec='white',fc='lightgrey',alpha=.9),ha="left", va="center")
        if p_direction == 'right':
            fig.text(.49,.65, 'Play direction', fontsize=14, bbox=dict(boxstyle="rarrow",ec='orange',fc='orange',alpha=.9),ha="left", va="center")
        else:
            fig.text(.49,.65, 'Play direction', fontsize=14, bbox=dict(boxstyle="larrow",ec='orange',fc='orange',alpha=.9),ha="left", va="center")
    ax.set_xlabel('')
    ax2.set_xlabel('')
    return fig;

### Now we reached the main function which is called `plot_player_mvt`  this function uses the previous functions to plot both player movement with direction, ball movment (if plotting a single play) and also heat map of player movment

In [ ]:
def plot_player_mvt(playername, season, week = None, playid= None, ball= False):
    '''
    plots player movements as scatter and heatmap in a whole year
    inputs:
    playername: string, the name of the player
    year: int (maybe required promptly), year in which tracking data was recorded
    week: the game week if it's required to plot only movement in a single game
    play: the play id if it's required to only plot a single play movements
    ball: bool, boolean to plot the ball used only when plotting a game or a play
    return:
    heatmap and scatter plots of a player's movement in a whole year
    '''
    seasons = [2018,2019,2020]
    if season not in seasons:
        while True:
            print(colored('Please make sure that selected season is in [2018:2020]','red'))
            season = input('Please select the year in which player\'s tracking data was in, kindly select one of the following\n1 for 2018\n2 for 2019\n3 for 2020\n4 to exit\n')
            if season in ['1','2','3']:
                season = int(season) + 2017
                break
            elif season == '4':
                return None;
            else:
                print('kindly enter one of the above mentioned options!')

    if week != None:
        while True:
            if ((week > 17) or (week < 1)) or type(week) != int:
                print(colored('wrong week value!, please make sure that selected week is in [1:17]','red'))
                week = int(input('kindly insert week again:\n'))
            else:
                gameid = get_game_id(season, week, playername= playername)
                player_games = get_player_games(season,playername)
                if (gameid not in player_games) or (player_games == None):
                    print(colored('{} didn\'t play in this week/season'.format(season),'red'))
                    return None
                else:
                    break
   
    temp_tracking = eval('tracking'+str(season))
    if (week != None) and (playid != None):
        if playername in duplicated_names:
            nflid = int(input('this name is duplicated kindly enter the nflid of the player'))
            temp_df = temp_tracking[(temp_tracking.gameId == gameid) & (temp_tracking.nflId == nflid) & (temp_tracking.playId == playid)]
        else:
            temp_df = temp_tracking[(temp_tracking.gameId == gameid) & (temp_tracking.displayName == playername) & (temp_tracking.playId == playid)]
        if len(temp_df) == 0:
            print(colored(playername +' doesn\'t have tracking data in {} week {} play no. {}!'.format(season, week, playid),'red'))
            return None;
        fig = plot_custom(temp_df, playername, year= season, gameid= gameid, playid= playid, ball= ball)
        del temp_tracking, temp_df
        gc.collect()
    elif week !=None:
        if playername in duplicated_names:
            nflid = int(input('this name is duplicated kindly enter the nflid of the player'))
            temp_df = temp_tracking[(temp_tracking.gameId == gameid) & (temp_tracking.nflId == nflid)]
        else:
            temp_df = temp_tracking[(temp_tracking.gameId == gameid) & (temp_tracking.displayName == playername)]
        if len(temp_df) == 0:
            print(colored(playername +' doesn\'t have tracking data in {} week {}!'.format(season, week),'red'))
            return None;
        fig = plot_custom(temp_df, playername, year= season, gameid= gameid, ball= ball)
        del temp_tracking, temp_df
        gc.collect()
    else:
        if playername in duplicated_names:
            nflid = int(input('this name is duplicated kindly enter the nflid of the player'))
            temp_df = temp_tracking[temp_tracking.nflId == nflid]
        else:
            temp_df = temp_tracking[temp_tracking.displayName == playername]
        if len(temp_df) == 0:
            print(colored(playername +' doesn\'t have tracking data in {} season!'.format(season),'red'))
            return None;
        temp_df = temp_tracking[temp_tracking.displayName == playername]
        fig = plot_custom(temp_df, playername, year= season, ball= ball)
        del temp_tracking, temp_df
        gc.collect()

### Now lets make an animation function called `plot_play` that make a video animation of a play with all players and also the ball 

In [ ]:
def plot_play(season, gameid, playid):
    """
    this function plot an animation of a whole play including all players
    inputs:
    gameid: unique game identifier
    playid: unique play identifier
    return:
    an animation of a play
    """
    fig, ax = plt.subplots(figsize=(1.2*(12),1.2*(5.4)))
    plt.rcParams["animation.html"] = "jshtml"
    tracking = eval('tracking'+str(season))
    eball = eval('ball_'+str(season)[-2:]+'_df')
    temp_tr = tracking[(tracking.playId ==playid) & (tracking.gameId == gameid)].set_index(['time','frameId' ]).sort_index()
    temp_ball = eball[(eball.playId ==playid) & (eball.gameId == gameid)].set_index(['time','frameId']).sort_index()
    frames = temp_ball.index                                                                

    pos_team =  plays[(plays.gameId == gameid) & (plays.playId == playid)].possessionTeam.values[0]
    home = games.loc[gameid,'homeTeamAbbr']
    away = games.loc[gameid,'visitorTeamAbbr']
    gtype = plays[(plays.gameId == gameid) & (plays.playId == playid)].specialTeamsPlayType.values[0]
    result = plays[(plays.gameId == gameid) & (plays.playId == playid)].specialTeamsResult.values[0]
    week = games.loc[gameid,'week']
    ax = football_field(ax)
    if home == pos_team:
        Att, Def = home, away
        temp_att = temp_tr[temp_tr.team == 'home']
        temp_def = temp_tr[temp_tr.team == 'away'] 
    else:
        Att, Def = away, home
        temp_def = temp_tr[temp_tr.team == 'home']
        temp_att = temp_tr[temp_tr.team == 'away'] 
    
    def animate(f):
        plt.cla()    
        ax = plt.gca()
        temp_ball_s = temp_ball.loc[f,:]
        ax.scatter(temp_ball_s.x, temp_ball_s.y, color='brown', alpha=1, s=220,zorder=8, label='Ball')
        temp_att_s = temp_att.loc[f,:]
        temp_def_s = temp_def.loc[f,:]
        ax.quiver(temp_att_s.x, temp_att_s.y, temp_att_s.x, temp_att_s.y, zorder=5, angles= (90- temp_att_s.dir ),
                  pivot="mid",edgecolor='none', color='red',alpha=.6,headwidth = 7, label= 'Att' )
        ax.quiver(temp_def_s.x, temp_def_s.y, temp_def_s.x, temp_def_s.y, zorder=5, angles= (90- temp_def_s.dir ),
                  pivot="mid",edgecolor='none', color='blue',alpha=.6,headwidth = 7,  label= 'Def')
        ax = football_field(ax)
        ax.text(0,.96,str(f[0]),fontsize=14, bbox=dict(boxstyle="round",ec='white',fc='white',alpha=.7),ha="left", va="center")
        ax.set_title('Game  {} | {} vs. {} | Ball with {}\nPlay type: {} | Result: {}'.format(week, home, away, pos_team, gtype, result),fontsize=18)
        ax.legend(loc= 'upper right',fontsize= 13,ncol=1)
        print('█',end='')     
    ani = matplotlib.animation.FuncAnimation(fig, animate, frames= frames,blit=False)
    html = HTML(ani.to_jshtml())
    display(html)
    plt.close()


In [ ]:
plot_player_mvt('Keishawn Bierria',2018)

In [ ]:
plot_player_mvt('Keishawn Bierria', 2019, week=17)

In [ ]:
plot_player_mvt('Keishawn Bierria', 2019, week=17,playid=36, ball= True)

In [ ]:
plot_player_mvt('Keishawn Bierria',2020)

### Testing the animation function

### First we get a list of all New York Giantshome games 

In [ ]:
nyg_home_games = get_team_games('NYG',season=2018, where='home')

### Getting plays in the first game where possession is with the Giants

In [ ]:
giants_plays = get_plays(nyg_home_games[0],posteam='NYG')

In [ ]:
plot_play(2018, nyg_home_games[0], giants_plays[0])

In [ ]:
plot_play(2018, 2018090906, 3527)

<a id='ml'></a>

# Machine learning starter

### This part is just a ML playground to explore some features importance


### First we make some aggregation by each play but we will separate the offensive from the defensive sides

In [ ]:
trackking_plays_agg = tracking_agg.groupby(['gameId','playId','side',
                      'specialTeamsPlayType',
                      'month','yards_to_endzone',
                      'yardsToGo', 'down','quarter',
                      'gameClock','pos_score_def','specialTeamsResult']).agg({'TAVG': 'mean','highcm': 'mean', 'age': 'mean','weight':'mean'}).reset_index()
trackking_plays_agg.gameClock = trackking_plays_agg.gameClock.apply(lambda x: round((int(x[:2]) + (int(x[3:5]) /60)),3)) 
trackking_plays_agg.sample(8)

### Now we split and make a function to
- train the model
- show training and test metrics
- show features importance
- plot confusion matrix

In [ ]:
X = pd.get_dummies(trackking_plays_agg[(trackking_plays_agg['side'] == 'offense') & (trackking_plays_agg['specialTeamsPlayType'] == 'Punt')][[ 'month',
       'yards_to_endzone', 'yardsToGo', 'down', 'quarter', 'gameClock',
       'pos_score_def','TAVG', 'highcm', 'age', 'weight']])
y = trackking_plays_agg[(trackking_plays_agg['side'] == 'offense') & (trackking_plays_agg['specialTeamsPlayType'] == 'Punt')]['specialTeamsResult']

In [ ]:
X_train, X_test, y_train, y_test = split(X, y, test_size=0.33, random_state=42)

In [ ]:
def plot_model(model, x_tr, y_tr,  x_ts, y_ts):
    model.fit(x_tr, y_tr)
    pred = model.predict(x_tr)
    print('train scores\naccuracy: ',accuracy_score(y_tr, pred),
      '\nprecision: ',precision_score(y_tr, pred, average='weighted'),
      '\nrecall: ',recall_score(y_tr, pred, average='weighted'),
      '\nf1: ',f1_score(y_tr, pred, average='weighted'))
    predt = model.predict(x_ts)
    print('test scores\naccuracy: ',accuracy_score(y_ts, predt),
      '\nprecision: ',precision_score(y_ts, predt, average='weighted'),
      '\nrecall: ',recall_score(y_ts, predt, average='weighted'),
      '\nf1: ',f1_score(y_ts, predt, average='weighted'))
    contr = confusion_matrix(pred, y_tr)
    conts = confusion_matrix(predt, y_ts)
    classeS = list(model.classes_)
    fig, (ax1 ,ax2) = plt.subplots(1,2, figsize= (24,8))
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=.1)
    sns.heatmap( contr, cbar=False, annot= True,fmt='1.0f',cmap='magma_r',  ax=ax1)
    ax1.set_xlabel('Predictied', fontsize=14)
    ax1.set_xticklabels(classeS,rotation=345)
    ax1.set_ylabel('Actual', fontsize=14)
    sns.heatmap(conts, cbar=False, annot= True,fmt='1.0f',cmap='magma_r', ax=ax2)
    ax2.set_xlabel('Predictied', fontsize=14)
    ax2.set_ylabel('Actual', fontsize=14)
    ax2.set_xticklabels(classeS,rotation=345)
    ax1.set_title('Train confusion matrix', fontsize=14)
    ax2.set_title('Test confusion matrix', fontsize=14)
    imp= pd.DataFrame(model.feature_importances_, index= X_train.columns ,columns=('imp',))
    print(imp.imp.sort_values(ascending = False))

In [ ]:
xgbclf= xgb.XGBClassifier(booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='merror',
              gamma=1,importance_type='gain',learning_rate=0.05,max_depth=4,n_estimators=600, n_jobs=4,
              objective='multi:softprob', random_state=0,reg_alpha=1, reg_lambda=1,tree_method='exact')
plot_model(xgbclf,X_train,y_train, X_test, y_test)

<center><img src="https://media4.giphy.com/media/q9TdZOZfsSspi/giphy.gif"></center>

### Not Much out there but as expected yards to end zone is importat feature 

<h2><center><font color='red'>Upvote this or Chuck will pay you a visit..</font></center></h2>
<center><img src="https://i.imgur.com/lDkgwsV.gif"></center>